# Basic Idea

Stretch out algorithm....

In [ ]:
from de.mindscan.fluentgenesis.bpe.bpe_model import BPEModel
from de.mindscan.fluentgenesis.bpe.bpe_encoder_decoder import SimpleBPEEncoder

In [ ]:
# load the BPE Model description file and hyper-parameter file.
model = BPEModel("16K-full","D:\\Projects\\SinglePageApplication\\Angular\\FluentGenesis-Classifier\\src\\de\\mindscan\\fluentgenesis\\bpe\\")
model.load_hparams()

# load associated vocabulary and bpe-pairs
model_vocabulary = model.load_tokens()
model_bpe_data = model.load_bpe_pairs()
    
# we must also make use of the vocabulary and the byte-pair occuences and pass that information to the encoder.
bpe_encoder = SimpleBPEEncoder(model_vocabulary, model_bpe_data)

## Case Number 0x01, this has substitutions and insertions

* "tt" became "span"
* "text-monosoace" was "added"


In [ ]:
del_line = '<tt class="ml-2 small">{{revision.shortrev}}</tt>'
add_line = '<span class="ml-2 small text-monospace">{{revision.shortrev}}</span>'

In [ ]:
bpe_del_line = bpe_encoder.encode([del_line])
bpe_add_line = bpe_encoder.encode([add_line])

In [ ]:
print("del_line:")
print(bpe_del_line)
print("add_line:")
print(bpe_add_line)


In [ ]:
bpe_del_line_stretched_expected=[
    61, 3397, 2839, 1756, 539, 46, 51, 2119, 110, 625,    0,  0,    0,   0,    0, 
    10003, 124, 124, 6778, 47, 1755, 6844, 126, 126, 1794, 3397, 63]
bpe_add_line_stretched_expected=[
    61, 3039, 2839, 1756, 539, 46, 51, 2119, 110, 625, 7645, 46, 2339, 450, 1070, 
    10003, 124, 124, 6778, 47, 1755, 6844, 126, 126, 1794, 3039, 63]

In [ ]:
def bpe_check_line(line_name, result, expected ):
    if(result != expected):
        print(str(line_name) + " - not good")
    else:
        print(str(line_name) + " - is good")
        

In [ ]:
def bpe_stretch_out(del_line: str, add_line: str):
    result_del_line = []
    result_add_line = []
    
    
    
    return (result_del_line, result_add_line)

bpe_del_line_stretched, bpe_add_line_stretched = bpe_stretch_out(bpe_del_line, bpe_add_line)

bpe_check_line("del_line",  bpe_del_line_stretched, bpe_del_line_stretched_expected)
bpe_check_line("add_line",  bpe_add_line_stretched, bpe_add_line_stretched_expected)


## Stretchout version 1

This works for the substitutions and insertions. in the previous example.

In [ ]:
def v1_find_relative(bpe_del_line,bpe_add_line):
    length_del = len(bpe_del_line)
    rel_find = [None]*length_del
    for i in range(0,length_del):
        element = bpe_del_line[i]
        if( element in bpe_add_line ):
            # actually it returns the first index, but actually we might want one of the later.
            index_inAdd = bpe_add_line.index(element)-i
            rel_find[i]=index_inAdd
    return rel_find


def v1_stretchout(bpe_del_line, rel_find):
    start_offset = 0;
    stretched=[]
    for i in range(0,len(bpe_del_line)):
        if rel_find[i] is None:
            stretched.append(bpe_del_line[i])
        elif rel_find[i] <= start_offset:
            stretched.append(bpe_del_line[i])
        else:
            stretched.extend([0]*(rel_find[i]-start_offset))
            stretched.append(bpe_del_line[i])
            # this is actually bad in case things were moved around...
            start_offset=rel_find[i]
    return stretched

stretchedv1 = v1_stretchout(bpe_del_line, v1_find_relative(bpe_del_line, bpe_add_line))

In [ ]:
bpe_check_line("addline",stretchedv1,bpe_del_line_stretched_expected)



print(stretchedv1)
print(bpe_del_line_stretched_expected)